In [ ]:
%load_ext lab_black
import pandas as pd
import numpy as np
import helper

In [ ]:
g = helper.parse_mikenet_sims("all_grain_fillIn.txt", "grain")
s = helper.parse_mikenet_sims("all_strain_fillIn.txt", "strain")
new_runs = pd.concat([g, s], ignore_index=True)
new_runs.sample(3)

In [ ]:
# check h-params
for x in ["Hidden", "Pnoise", "Epsilon", "PhoHid"]:
    print(f"{x}: {new_runs[x].unique()}")

In [ ]:
helper.count_grid(new_runs, ["Hidden", "Pnoise", "Epsilon", "PhoHid"], "raw")

In [ ]:
# Read canonical df (1520)
df_1520 = pd.read_csv("../sims/1520_sims.csv", index_col=0)
df_1520.sample(3)

In [ ]:
# Read canonical df (1695)
df_1695 = pd.read_csv("../sims/1695_sims.csv", index_col=0)
df_1695.sample(3)

In [ ]:
np.intersect1d(df_1695.ID.unique(), new_runs.ID.unique())
# There are duplicate between 1695 and new_runs, double check content

In [ ]:
df_1695.loc[
    df_1695.ID == 166394835,
]

In [ ]:
new_runs.loc[
    new_runs.ID == 166394835,
]

- Duplicate ID has identical content
- Merge into one last canonical version df_1520

In [ ]:
# Check duplicate
x = df_1520.ID.unique()
y = new_runs.ID.unique()
np.intersect1d(x, y)

In [ ]:
len(y) + 1520

In [ ]:
# Make new canonical df
df_2395 = pd.concat([df_1520, new_runs], ignore_index=True)
df_2395.to_csv("../sims/2395_sims.csv")

In [ ]:
helper.count_grid(df_2395, ["Hidden", "PhoHid", "Pnoise", "Epsilon"], "raw")
# Min at needed space n = 7 

### Proper grid (n=525)

In [ ]:
# Make a ID only h-param dataset
v = ["ID", "Hidden", "PhoHid", "Pnoise", "Epsilon"]
hpar = df_2395[v].groupby("ID").mean().reset_index()
hpar["Epsilon"] = hpar.Epsilon.round(3)  # Fix Epsilon rounding problem after mean

In [ ]:
# Get 7 ID from each cell in desired grid
n = 7
df = pd.DataFrame()

for hidden in [25, 50, 75, 100, 150, 200, 250]:
    for p_noise in [0, 2, 4, 6, 8]:
        for epsilon in [0.002, 0.004, 0.006, 0.008, 0.010]:
            tmp = hpar.loc[
                (hpar.Hidden == hidden)
                & (hpar.Pnoise == p_noise)
                & (hpar.Epsilon == epsilon)
                & (hpar.PhoHid == 20)
            ]

            df = df.append(tmp.sample(n), ignore_index=True)

In [ ]:
# Check len == n * noise_lvs * hidden_lvs * lr_lvs
assert len(df) == 7 * 7 * 5 * 5
print(len(df))

In [ ]:
sel = df_2395.loc[
    df_2395.ID.isin(df.ID),
]

In [ ]:
sel.to_csv("../sims/part3_1225.csv")